In [1]:
import argparse
import os
import sys
from pathlib import Path
import json
import matplotlib.pyplot as plt
%matplotlib inline

import imageio
import numpy as np
from tqdm import tqdm
try:
    import renderpy
except ImportError:
    print("renderpy not installed. Please install renderpy from https://github.com/liu115/renderpy")
    sys.exit(1)

In [2]:
from utils.colmap import read_model, write_model, Image
from scene_release import ScannetppScene_Release
from utils.utils import run_command, load_yaml_munch, load_json, read_txt_list

In [3]:
p = argparse.ArgumentParser()
p.add_argument("config_file", help="Path to config file", default="/home/kumaraditya/scannetpp/common/configs/render.yml", nargs="?")
args = p.parse_args([])

print(f"Config file: {args.config_file}")


Config file: /home/kumaraditya/scannetpp/common/configs/render.yml


In [4]:
cfg = load_yaml_munch(args.config_file)

# get the scenes to process
if cfg.get("scene_ids"):
    scene_ids = cfg.scene_ids
elif cfg.get("splits"):
    scene_ids = []
    for split in cfg.splits:
        split_path = Path(cfg.data_root) / "splits" / f"{split}.txt"
        scene_ids += read_txt_list(split_path)

output_dir = cfg.get("output_dir")
if output_dir is None:
    # default to data folder in data_root
    output_dir = Path(cfg.data_root) / "data"
output_dir = Path(output_dir)

render_devices = []
if cfg.get("render_dslr", False):
    render_devices.append("dslr")
    raise Exception("This code is has not been tested with the DSLR data.")
if cfg.get("render_iphone", False):
    render_devices.append("iphone")

In [5]:
scene_id = scene_ids[0]
print(scene_id)
print(render_devices)

c0f5742640
['iphone']


In [6]:
from render_crops_utils import vert_to_obj_lookup, CropHeap, crop_rgb_mask, plot_grid_images

scene = ScannetppScene_Release(scene_id, data_root=Path(cfg.data_root) / "data")
render_engine = renderpy.Render()
render_engine.setupMesh(str(scene.scan_mesh_path))

# Load annotations
segments_anno = json.load(open(scene.scan_anno_json_path, "r"))
n_objects = len(segments_anno["segGroups"])
instance_colors = np.random.randint(low=0, high=256, size=(n_objects + 1, 3), dtype=np.uint8)
instance_colors[0] = 255 # White bg
vert_to_obj = vert_to_obj_lookup(segments_anno)

# Crop heaps
crop_heaps = dict()
for obj in segments_anno["segGroups"]:
    crop_heaps[obj["id"]] = dict()
    crop_heaps[obj["id"]]["label"] = obj["label"]
    crop_heaps[obj["id"]]["heap"] = CropHeap(max_size=4)

# Background class is 0
assert 0 not in crop_heaps
crop_heaps[0] = dict()
crop_heaps[0]["label"] = "BACKGROUND"
crop_heaps[0]["heap"] = CropHeap(max_size=4)


for device in render_devices:
    if device == "dslr":
        cameras, images, points3D = read_model(scene.dslr_colmap_dir, ".txt")
    else:
        cameras, images, points3D = read_model(scene.iphone_colmap_dir, ".txt")
    assert len(cameras) == 1, "Multiple cameras not supported"
    camera = next(iter(cameras.values()))

    fx, fy, cx, cy = camera.params[:4]
    params = camera.params[4:]
    camera_model = camera.model
    render_engine.setupCamera(
        camera.height, camera.width,
        fx, fy, cx, cy,
        camera_model,
        params,      # Distortion parameters np.array([k1, k2, k3, k4]) or np.array([k1, k2, p1, p2])
    )

    near = cfg.get("near", 0.05)
    far = cfg.get("far", 20.0)
    rgb_dir = Path(cfg.output_dir) / scene_id / device / "render_rgb"
    depth_dir = Path(cfg.output_dir) / scene_id / device / "render_depth"
    # crop_dir = Path(cfg.output_dir) / scene_id / device / "render_crops_kumar_w_sam"
    rgb_dir.mkdir(parents=True, exist_ok=True)
    depth_dir.mkdir(parents=True, exist_ok=True)
    # crop_dir.mkdir(parents=True, exist_ok=True)

    for _, image in tqdm(images.items(), f"Rendering object crops using {device} images"):
        world_to_camera = image.world_to_camera

        rgb_rendered, _, vert_indices = render_engine.renderAll(world_to_camera, near, far)

        iphone_rgb_path = Path(scene.iphone_rgb_dir) / image.name
        rgb = np.asarray(imageio.imread(iphone_rgb_path))

        vert_instance = vert_to_obj[vert_indices]
        pix_instance = vert_instance[:, :, 0] # Some triangles actually belong to different objects. I don't think it will matter for crops.

        # Visualize instances
        # instance_rgb = instance_colors[pix_instance]
        # imageio.imwrite(rgb_dir / image.name, instance_rgb)

        objs = np.unique(pix_instance)

        for obj in objs:
            mask = pix_instance == obj
            crop = crop_rgb_mask(rgb, rgb_rendered, mask, inflate_px=100)
            crop_heaps[obj]["heap"].push(crop)


        # instance_rgb = instance_rgb.astype(np.uint8)
        # # Make depth in mm and clip to fit 16-bit image
        # depth = (depth.astype(np.float32) * 1000).clip(0, 65535).astype(np.uint16)
        # depth_name = image.name.split(".")[0] + ".png"
        # imageio.imwrite(depth_dir / depth_name, depth)

Init EGL
Detected 5 devices
Using device 0
Using EGL version 1.5
OpenGL version: 4.6.0 NVIDIA 550.107.02
EGL version: 1.5
Loaded mesh:MeshData:
	Vertices:  917079
	Colors:    917079
	Normals:   0
	TexCoords: 0


Copy mesh to GPU: 917079 vertices, 1832073 faces
Setup the frame and render buffer


Rendering object crops using iphone images:   0%|          | 0/586 [00:00<?, ?it/s]/tmp/ipykernel_242573/1539412114.py:61: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  rgb = np.asarray(imageio.imread(iphone_rgb_path))
Rendering object crops using iphone images: 100%|██████████| 586/586 [03:52<00:00,  2.53it/s]


In [7]:
# for id, entry in tqdm(crop_heaps.items(), f"Rendering image grids"):
#     heap = entry["heap"]
#     label = entry["label"]
#     if len(heap) and label.lower() not in [
#         "background",
#         "wall",
#         "floor",
#         "ceiling",
#         "split",
#         "remove",
#     ]:
#         crops = heap.get_sorted()
#         rgbs = [c.rgb for c in crops]
#         masks = [c.mask for c in crops]
#         scores = [c.score for c in crops]
#         plot_grid_images(
#             rgbs + masks, grid_width=len(rgbs), title=entry["label"]
#         )
#         plt.savefig(crop_dir / f"{str(id).zfill(5)}.jpg")
#         plt.close()

In [24]:
import torch
import os
import tempfile
import shutil
import cv2
import numpy as np
import logging
import random

from sam2.build_sam import build_sam2, build_sam2_video_predictor
from sam2.sam2_image_predictor import SAM2ImagePredictor

# Set up logging
logging.basicConfig(
    level=logging.INFO,  # Logging level
    format="%(asctime)s - %(levelname)s - %(message)s",  # Log format
    filename="sam2_model.log",  # Log file path
    filemode="w",  # 'w' to overwrite the log file each time, 'a' to append
)


class SAM2VideoMaskModel:
    def __init__(
        self, sam2_checkpoint, model_cfg, num_points=5, device="cuda", temp_dir=None
    ):
        """
        Initialize SAM2 model and set device.
        """
        self.device = torch.device(device)
        if self.device.type == "cuda":
            torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

        self.predictor = build_sam2_video_predictor(
            model_cfg, sam2_checkpoint, device=self.device
        )

        # Placeholder for rgbs, masks, masks_padded, rgbs_padded, padding information, and scores
        self.rgbs = []
        self.rgbs_padded = []
        self.masks = []
        self.masks_padded = []
        self.padding_info = []
        self.scores = []

        if temp_dir is None:
            self.temp_dir = tempfile.mkdtemp()
        else:
            self.temp_dir = temp_dir
        logging.info(f"Temporary directory created at: {self.temp_dir}")

        self.num_points = num_points
        self.predictor_inference_state = None

    def store_data(self, rgbs, masks, scores):
        """
        Pads the RGB images and masks to the size of the largest image and stores them.
        """
        if len(rgbs) != len(masks) or len(rgbs) != len(scores):
            raise ValueError("The number of RGB images, masks, and scores must match.")

        max_h, max_w = self._get_max_dimensions(rgbs)

        for idx, (rgb, mask, score) in enumerate(zip(rgbs, masks, scores)):
            padded_rgb, padded_mask, padding_info = self._pad_image_and_mask(
                rgb, mask, max_h, max_w
            )
            self._store_padded_data(
                idx, rgb, padded_rgb, mask, padded_mask, score, padding_info
            )

        self.predictor_inference_state = self._initialize_inference_state()

    def refine_masks_and_propagate(self, mode=None):
        """
        Refine masks using SAM2 predictor.
        """
        if self.predictor_inference_state is None:
            raise ValueError(
                "Inference state not initialized. Please store data before refining masks."
            )

        if mode == "points":
            points, labels, highest_score_idx = self._get_initial_prompts(points=True)
            _ = self.refine_mask_w_points_prompt(points, labels, highest_score_idx)
            refined_masks = self.propagate_prompt()
            self.masks = refined_masks
            return refined_masks

        elif mode == "mask":
            highest_score_mask, highest_score_idx = self._get_initial_prompts(mask=True)
            _ = self.refine_mask_w_mask_prompt(highest_score_mask, highest_score_idx)
            refined_masks = self.propagate_prompt()
            self.masks = refined_masks
            return refined_masks

        else:
            raise ValueError("Mode must be either 'points' or 'mask'.")

    def reset_inference_state(self):
        """
        Reset the inference state to the initial state.
        """
        self.predictor.reset_state(self.predictor_inference_state)
        self.predictor_inference_state = self._initialize_inference_state()

    def cleanup(self):
        """
        Clean up the temporary directory and clear stored data.
        """
        self.predictor.reset_state(self.predictor_inference_state)
        self.predictor_inference_state = None
        self._clear_temp_directory()
        self._clear_storage()

    def refine_mask_w_points_prompt(self, points, labels, frame_idx):
        """
        Refine the masks with point prompts.
        """
        _, _, out_mask_logits = self.predictor.add_new_points_or_box(
            inference_state=self.predictor_inference_state,
            frame_idx=frame_idx,
            obj_id=1,
            points=points,
            labels=labels,
            box=None,
        )

        refined_mask = (out_mask_logits[0] > 0.0).cpu().numpy().squeeze()
        unpadded_refined_mask = self._unpad_mask(refined_mask, frame_idx)

        return unpadded_refined_mask

    def refine_mask_w_mask_prompt(self, highest_score_mask, frame_idx):
        """
        Refine the masks using the mask with the highest score.
        """
        _, _, out_mask_logits = self.predictor.add_new_mask(
            inference_state=self.predictor_inference_state,
            frame_idx=frame_idx,
            obj_id=1,
            mask=highest_score_mask,
        )

        refined_mask = (out_mask_logits[0] > 0.0).cpu().numpy().squeeze()
        unpadded_refined_mask = self._unpad_mask(refined_mask, frame_idx)

        return unpadded_refined_mask

    def propagate_prompt(self):
        """
        Propagate the prompt through the video.
        """
        refined_masks = []

        for (
            out_frame_idx,
            out_obj_ids,
            out_mask_logits,
        ) in self.predictor.propagate_in_video(self.predictor_inference_state):
            refined_mask = (out_mask_logits[0] > 0.0).cpu().numpy().squeeze()
            unpadded_refined_mask = self._unpad_mask(refined_mask, out_frame_idx)

            if np.sum(refined_mask) == 0:
                logging.warning(
                    f"Refined mask is empty for frame {out_frame_idx}. Using the old mask."
                )
                refined_mask = self.masks_padded[out_frame_idx]

            refined_masks.append(unpadded_refined_mask)

        if len(refined_masks) < len(self.masks):
            raise ValueError(
                "The number of refined masks is less than the number of input masks."
            )

        return refined_masks

    def _get_max_dimensions(self, rgbs):
        """
        Get the maximum height and width among the provided RGB images.
        """
        max_h = max([rgb.shape[0] for rgb in rgbs])
        max_w = max([rgb.shape[1] for rgb in rgbs])
        return max_h, max_w

    def _pad_image_and_mask(self, rgb, mask, max_h, max_w):
        """
        Pads the RGB and mask to the provided max dimensions.
        """
        h, w, _ = rgb.shape
        pad_h, pad_w = max_h - h, max_w - w
        padding_info = ((0, pad_h), (0, pad_w))

        padded_rgb = np.pad(
            rgb, ((0, pad_h), (0, pad_w), (0, 0)), mode="constant", constant_values=0
        )
        padded_mask = np.pad(
            mask, ((0, pad_h), (0, pad_w)), mode="constant", constant_values=0
        )

        return padded_rgb, padded_mask, padding_info

    def _store_padded_data(
        self, idx, rgb, padded_rgb, mask, padded_mask, score, padding_info
    ):
        """
        Store padded data, and save the padded RGB image to the temp directory.
        """
        rgb_filename = os.path.join(self.temp_dir, f"{idx}.jpg")
        cv2.imwrite(rgb_filename, padded_rgb)

        self.rgbs.append(rgb)
        self.rgbs_padded.append(padded_rgb)
        self.masks.append(mask)
        self.masks_padded.append(padded_mask)
        self.scores.append(score)
        self.padding_info.append(padding_info)

    def _initialize_inference_state(self):
        """
        Initialize the predictor's inference state.
        """
        return self.predictor.init_state(video_path=self.temp_dir)

    def _get_initial_prompts(self, points=False, mask=False):
        """
        Determine initial points based on the mask with the highest score.
        """
        highest_score_idx = np.argmax(self.scores)
        highest_score_mask = self.masks_padded[highest_score_idx]

        if highest_score_idx != 0:
            logging.info(
                f"Using mask with the highest score from frame {highest_score_idx} as the initial prompt."
            )

        if points:
            mask_indices = np.argwhere(highest_score_mask > 0)
            points = np.array(
                [
                    mask_indices[np.random.choice(len(mask_indices))]
                    for _ in range(self.num_points)
                ],
                dtype=np.float32,
            )
            labels = np.ones(self.num_points, dtype=np.int32)

            return points, labels, highest_score_idx
        elif mask:
            return highest_score_mask, highest_score_idx
        else:
            raise ValueError("Either points or mask must be True.")

    def _unpad_mask(self, mask, frame_idx):
        """
        Unpad a single mask based on its padding information.
        """
        pad_h, pad_w = self.padding_info[frame_idx]

        # Unpad the mask based on padding info
        unpadded_mask = mask
        if pad_h[1] > 0:  # Check if there was padding at the bottom
            unpadded_mask = unpadded_mask[: -pad_h[1], :]
        if pad_w[1] > 0:  # Check if there was padding at the right
            unpadded_mask = unpadded_mask[:, : -pad_w[1]]

        return unpadded_mask

    def _clear_temp_directory(self):
        """
        Clears the contents of the temporary directory but leaves the directory itself.
        """
        if os.path.exists(self.temp_dir):
            for filename in os.listdir(self.temp_dir):
                file_path = os.path.join(self.temp_dir, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)  # Remove file or symbolic link
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)  # Remove subdirectory and its contents
                except Exception as e:
                    logging.error(f"Failed to delete {file_path}. Reason: {e}")

            logging.info(f"Contents of temporary directory {self.temp_dir} removed.")

    def _clear_storage(self):
        """
        Clears all stored data except for the SAM2 model.
        """
        self.rgbs = []
        self.rgbs_padded = []
        self.masks = []
        self.masks_padded = []
        self.scores = []
        self.padding_info = []
        logging.info(
            "Cleared all stored images, masks, scores, and padding information."
        )


class SAM2ImageMaskModel:
    def __init__(
        self,
        sam2_checkpoint,
        model_cfg,
        device="cuda",
        num_points=5,
        ransac_iterations=10,
    ):
        """
        Initialize SAM2 model, set device, and configure number of points and RANSAC iterations.
        """
        self.device = torch.device(device)
        self.num_points = num_points  # Number of points to sample from the mask
        self.ransac_iterations = ransac_iterations  # Number of RANSAC iterations

        if self.device.type == "cuda":
            torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
            if torch.cuda.get_device_properties(0).major >= 8:
                torch.backends.cuda.matmul.allow_tf32 = True
                torch.backends.cudnn.allow_tf32 = True

        self.predictor = self._build_predictor(sam2_checkpoint, model_cfg)
        self._initialize_storage()

    def store_data(self, rgbs, masks, scores):
        """
        Stores the RGB images, masks, and scores.
        """
        if len(rgbs) != len(masks) or len(rgbs) != len(scores):
            raise ValueError("The number of RGB images, masks, and scores must match.")

        for idx, (rgb, mask, score) in enumerate(zip(rgbs, masks, scores)):
            self._store_data(idx, rgb, mask, score)

    def refine_masks(self):
        """
        Public method to trigger RANSAC-based mask refinement.
        """
        masks_refined, sam_scores = self._ransac_mask_selection()
        self.masks = masks_refined
        self.scores = sam_scores
        return masks_refined, sam_scores

    def refine_mask_w_point_prompt(self, frame_idx, points, labels):
        """
        Refine the mask for the provided frame index.
        """
        rgb = self.rgbs[frame_idx]
        bbox = None

        predicted_mask, predicted_scores = self._predict_mask(rgb, points, bbox, labels)

        self.masks_refined[frame_idx] = predicted_mask[0]
        self.scores[frame_idx] = predicted_scores[0]

        return predicted_mask[0], predicted_scores[0]

    def cleanup(self):
        """
        Cleans up the stored data, including RGBs, masks, and scores,
        while keeping the SAM model ready for further use.
        """
        # Clear all stored data (RGBs, masks, scores)
        self.rgbs = []
        self.masks = []
        self.scores = []

        logging.info("Cleared all stored images, masks, scores, and refined masks.")

    def _ransac_mask_selection(self):
        """
        Perform RANSAC-like sampling of points and select the best mask based on SAM score.
        """
        masks_refined = []
        sam_scores = []

        for idx, (rgb, mask) in enumerate(zip(self.rgbs, self.masks)):
            best_score = -float("inf")
            best_mask = None

            for _ in range(self.ransac_iterations):
                try:
                    # Sample points from the mask
                    sampled_points = self._sample_points_from_mask(mask)
                    labels = np.ones(len(sampled_points), dtype=np.int32)
                    bbox = self._get_bounding_box_from_mask(mask)

                    # Get mask prediction and score from SAM, assume single mask and score
                    predicted_mask, predicted_scores = self._predict_mask(
                        rgb, sampled_points, bbox, labels
                    )

                    predicted_mask = predicted_mask[0]

                    score = predicted_scores[0]  # Assuming single mask is returned
                    if score > best_score:
                        best_score = score
                        best_mask = predicted_mask

                except ValueError as e:
                    logging.warning(f"Skipping frame {idx} due to error: {e}")
                    continue

            # Store the best mask and score for the current frame
            masks_refined.append(best_mask)
            sam_scores.append(best_score)

        return masks_refined, sam_scores

    def _build_predictor(self, sam2_checkpoint, model_cfg):
        """
        Helper function to build the SAM2 image predictor.
        """
        sam2 = build_sam2(model_cfg, sam2_checkpoint, device=self.device)
        return SAM2ImagePredictor(sam2)

    def _initialize_storage(self):
        """
        Initializes storage for RGB images, masks, and related data.
        """
        self.rgbs = []
        self.masks = []
        self.scores = []

    def _store_data(self, idx, rgb, mask, score):
        """
        Store RGB, mask, and score data.
        """
        self.rgbs.append(rgb)
        self.masks.append(mask)
        self.scores.append(score)

    def _sample_points_from_mask(self, mask):
        """
        Sample n points from the provided mask where the mask is non-zero.
        """
        mask_indices = np.argwhere(mask > 0)  # Get non-zero mask points
        if len(mask_indices) == 0:
            raise ValueError("No valid mask points to sample from.")

        # Randomly sample 'n' points
        sampled_points = np.array(
            random.choices(mask_indices, k=self.num_points), dtype=np.float32
        )
        return sampled_points

    def _predict_mask(self, rgb, points, bbox, labels):
        """
        Use sampled points to prompt SAM for mask prediction and return mask and score.
        """

        self.predictor.set_image(rgb)

        if bbox is not None:
            bbox = bbox[None, :]

        # Perform mask prediction using the points and labels
        masks, scores, _ = self.predictor.predict(
            point_coords=points,
            point_labels=labels,
            box=bbox,
            multimask_output=False,
        )

        if masks.dtype != bool:
            masks = masks.astype(bool)

        return masks, scores

    def _get_bounding_box_from_mask(self, mask):
        """
        Given a binary mask, return the bounding box in xyxy format.
        """

        # Find the indices where the mask is True (non-zero)
        rows, cols = np.where(mask)

        # If the mask is empty (no True values), return an empty bounding box
        if len(rows) == 0 or len(cols) == 0:
            return [0, 0, 0, 0]

        # Get the bounding box coordinates
        x_min = np.min(cols)
        y_min = np.min(rows)
        x_max = np.max(cols)
        y_max = np.max(rows)

        return np.array([x_min, y_min, x_max, y_max])


In [29]:
import logging

sam2_checkpoint = "/home/kumaraditya/checkpoints/sam2_hiera_large.pt"
sam2_model_cfg = "sam2_hiera_l.yaml"
sam2_video_model = SAM2VideoMaskModel(sam2_checkpoint, sam2_model_cfg)
sam2_img_model = SAM2ImageMaskModel(sam2_checkpoint, sam2_model_cfg, num_points=3, ransac_iterations=5)

def sam2_refine_masks(rgbs, masks, scores):
    sam2_img_model.store_data(rgbs, masks, scores)
    img_refined_masks, img_refined_scores = sam2_img_model.refine_masks()
    sam2_img_model.cleanup()

    sorted_data = sorted(zip(img_refined_scores, rgbs, img_refined_masks), key=lambda x: x[0], reverse=True)
    img_refined_scores, rgbs, img_refined_masks = zip(*sorted_data)

    # Convert them back to lists
    img_refined_scores = list(img_refined_scores)
    rgbs = list(rgbs)
    img_refined_masks = list(img_refined_masks)

    sam2_video_model.store_data(rgbs, img_refined_masks, img_refined_scores)
    video_refined_masks = sam2_video_model.refine_masks_and_propagate(mode="mask")
    sam2_video_model.cleanup()

    return rgbs, video_refined_masks, img_refined_scores

In [30]:
refined_crops_data = dict()

crop_dir = Path(cfg.output_dir) / scene_id / device / "render_crops_sam2"
crop_dir.mkdir(parents=True, exist_ok=True)

for id, entry in tqdm(crop_heaps.items(), f"Rendering image grids"):
    heap = entry["heap"]
    label = entry["label"]
    if len(heap) and label.lower() not in [
        "background",
        "wall",
        "floor",
        "ceiling",
        "split",
        "remove",
    ]:
        
        logging.info(f"Processing object id: {id} with label: {label}")
        
        crops = heap.get_sorted()
        rgbs = [c.rgb for c in crops]
        masks = [c.mask for c in crops]
        scores = [c.score for c in crops]

        rgbs, refined_masks, refined_scores = sam2_refine_masks(rgbs, masks, scores)

        refined_crops_data[id] = dict()
        refined_crops_data[id]["rgbs"] = rgbs
        refined_crops_data[id]["masks"] = refined_masks
        refined_crops_data[id]["scores"] = refined_scores
        refined_crops_data[id]["label"] = label

        #print all rgbs and masks shapes
        print("RGB shapes: ", [rgb.shape for rgb in rgbs])
        print("Mask shapes: ", [mask.shape for mask in refined_masks])
        
        plot_grid_images(
            rgbs, refined_masks, grid_width=len(rgbs), title=label
        )
        plt.savefig(crop_dir / f"{str(id).zfill(5)}.jpg")
        plt.close()

propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.84it/s]


RGB shapes:  [(385, 616, 3), (594, 607, 3), (580, 606, 3), (597, 601, 3)]
Mask shapes:  [(385, 616), (594, 607), (580, 606), (597, 601)]


frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 42.73it/s]it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.89it/s]


RGB shapes:  [(298, 439, 3), (295, 461, 3), (371, 385, 3), (389, 412, 3)]
Mask shapes:  [(298, 439), (295, 461), (371, 385), (389, 412)]


frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 43.51it/s]it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.88it/s]


...

frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 40.26it/s]s/it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.39it/s]


RGB shapes:  [(435, 536, 3), (405, 463, 3), (487, 491, 3), (468, 475, 3)]
Mask shapes:  [(435, 536), (405, 463), (487, 491), (468, 475)]


frame loading (JPEG): 100%|██████████| 4/4 [00:00<00:00, 31.56it/s]s/it]
/home/kumaraditya/segment-anything-2/sam2/sam2_video_predictor.py:873: UserWarning: cannot import name '_C' from 'sam2' (/home/kumaraditya/segment-anything-2/sam2/__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/segment-anything-2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(
propagate in video: 100%|██████████| 4/4 [00:01<00:00,  3.39it/s]


RGB shapes:  [(932, 1022, 3), (931, 1025, 3), (940, 1043, 3), (926, 1026, 3)]
Mask shapes:  [(932, 1022), (931, 1025), (940, 1043), (926, 1026)]


Rendering image grids: 100%|██████████| 135/135 [17:53<00:00,  7.95s/it]


In [31]:
import cv2

def save_crops_data(crops_data, output_dir, pad_length=5):
    rgb_dir = output_dir / "rgbs"
    mask_dir = output_dir / "masks"
    metadata_dir = output_dir / "metadata"
    
    os.makedirs(rgb_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)
    
    for obj_id, data in crops_data.items():
        padded_obj_id = str(obj_id).zfill(pad_length)

        for crop_id, (rgb, mask) in enumerate(zip(data["rgbs"], data["masks"])):
            rgb_filename = os.path.join(rgb_dir, f"{padded_obj_id}_{crop_id}_rgb.png")
            cv2.imwrite(rgb_filename, cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))  # Convert to BGR before saving

            mask_filename = os.path.join(mask_dir, f"{padded_obj_id}_{crop_id}_mask.npy")
            np.save(mask_filename, mask)

        scores = [round(float(score), 4) for score in data["scores"]]

        metadata = {
            "scores": scores,
            "label": data["label"]
        }
        metadata_filename = os.path.join(metadata_dir, f"{padded_obj_id}_metadata.json")
        with open(metadata_filename, 'w') as metadata_file:
            json.dump(metadata, metadata_file, indent=4)


In [32]:
crops_data_dir = Path(cfg.output_dir) / scene_id / device / "crops_data"
save_crops_data(refined_crops_data, crops_data_dir)